In [24]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', True)

In [25]:
monkeys= open('day11_input.txt').read().splitlines()

In [26]:
monkeys

['Monkey 0:',
 '  Starting items: 98, 97, 98, 55, 56, 72',
 '  Operation: new = old * 13',
 '  Test: divisible by 11',
 '    If true: throw to monkey 4',
 '    If false: throw to monkey 7',
 '',
 'Monkey 1:',
 '  Starting items: 73, 99, 55, 54, 88, 50, 55',
 '  Operation: new = old + 4',
 '  Test: divisible by 17',
 '    If true: throw to monkey 2',
 '    If false: throw to monkey 6',
 '',
 'Monkey 2:',
 '  Starting items: 67, 98',
 '  Operation: new = old * 11',
 '  Test: divisible by 5',
 '    If true: throw to monkey 6',
 '    If false: throw to monkey 5',
 '',
 'Monkey 3:',
 '  Starting items: 82, 91, 92, 53, 99',
 '  Operation: new = old + 8',
 '  Test: divisible by 13',
 '    If true: throw to monkey 1',
 '    If false: throw to monkey 2',
 '',
 'Monkey 4:',
 '  Starting items: 52, 62, 94, 96, 52, 87, 53, 60',
 '  Operation: new = old * old',
 '  Test: divisible by 19',
 '    If true: throw to monkey 3',
 '    If false: throw to monkey 1',
 '',
 'Monkey 5:',
 '  Starting items: 9

# Part 1 & 2

In [27]:
import operator

def run_formula(old, formula):
    elements = formula.split()
    oper = elements[3]
    value = old if elements[4]=="old" else int(elements[4])    
    ops = {
        '+' : operator.add,
        '-' : operator.sub,
        '*' : operator.mul,
        '/' : operator.truediv,  # use operator.div for Python 2
        '%' : operator.mod,
        '^' : operator.xor,
    }
    result = np.int64(ops[oper](old, value))
    return result

In [28]:
def create_monkey_dict():
    monkey_dict = {}

    for line in monkeys:
        if line[0:6] =="Monkey":
            current_monkey = int(line[7])
        if line[0:6] =='  Star':
            current_items = [int(x) for x in line[18:].split(sep=", ")]
        if line[0:6] =='  Oper':
            current_formula = line[13:]
        if line[0:6] =='  Test':
            current_test = int(line[21:])
        if line[0:10] == '    If tru':
            monkey_true = int(line[29:])
        if line[0:10] == '    If fal':
            monkey_false = int(line[30:])

        try:
            monkey_dict[current_monkey]={"items":current_items, 
                                         "formula":current_formula, 
                                         "test":current_test,
                                         "monkey_true":monkey_true,
                                         "monkey_false":monkey_false}
        except:
            continue
    return monkey_dict

In [29]:
def run_one_round(monkey_dict, monk_counter, mcd, part2):
    for monk, char in monkey_dict.items():
        monkey = monkey_dict[monk]
        for item in monkey["items"]:
            monk_counter[monk]+=1
            #print("monkey", monk, "item", item, "overall items", monkey["items"])
            #new worry level
            new = run_formula(item, monkey["formula"])
            #print("--new worry", new)
            #monkey gets bored
            new = np.floor(new/3) if part2 == False else new%mcd
            #run test
            #print("--new", new, "test", monkey["test"])
            if new % monkey["test"] == 0:
                #print("--give to monkey", monkey["monkey_true"])
                monkey_dict[monk]["items"] = [x for x in monkey_dict[monk]["items"] if x != item]
                monkey_dict[monkey["monkey_true"]]["items"].append(new)
                #print("--current monkey items", monk, monkey_dict[monk]["items"])
                #print("--monkey true items", monkey["monkey_true"], monkey_dict[monkey["monkey_true"]]["items"])
                
            else:
                #print("--give to monkey", monkey["monkey_false"])
                monkey_dict[monk]["items"] = [x for x in monkey_dict[monk]["items"] if x != item]
                monkey_dict[monkey["monkey_false"]]["items"].append(new)   
                #print("--current monkey items", monk, monkey_dict[monk]["items"])
                #print("--monkey false items", monkey["monkey_false"], monkey_dict[monkey["monkey_false"]]["items"])

    #print([monkey_dict[monk]["items"] for monk, char in monkey_dict.items()])
    return monk_counter

In [30]:
def run_n_rounds(n):
    monkey_dict=create_monkey_dict()
    mcd = np.prod([monkey_dict[monk]["test"] for monk, char in monkey_dict.items()])
    monk_counter={monk:0 for monk, char in monkey_dict.items()}
    for r in range(0, n):
        #print("--- Round", r+1)
        monk_counter = run_one_round(monkey_dict, monk_counter, mcd, True)

    print(monk_counter)
    return monk_counter

monk_counter = run_n_rounds(10000)

top2 = sorted(monk_counter.values())[-2:]
print("Final score", top2[0]*top2[1])

{0: 71076, 1: 35485, 2: 88637, 3: 106189, 4: 17918, 5: 109795, 6: 56795, 7: 120569}
Final score 13237873355
